In [3]:
import pandas as pd

# проверим кодировку файла

from chardet.universaldetector import UniversalDetector 
detector = UniversalDetector()
with open('VBA_Блок3_Данные.csv', 'rb') as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
detector.close()


{'encoding': 'windows-1251',
 'confidence': 0.6569243596460919,
 'language': 'Russian'}

In [303]:
# загрузим датасет

import numpy as np

db_path = pd.read_csv(
    'VBA_Блок3_Данные.csv',encoding = 'windows-1251',sep=';'
)


In [323]:
def matrix_func(db_path,bank,obligation):
    
    # импортируем библиотеку Pandas
    import pandas as pd
    
    # создаем копию файла 
    df = db_path.copy()
    
    # фильтруем датасет по банку и обязательствам
    df = df[
        (df['Первоначальный кредитор']==bank)&
        (df['Тип обязательства']==obligation)
    ]
    
    # приводим тип данных к datetime
    df['Дата'] = pd.to_datetime(df['Дата'],format = '%d/%m/%Y')
    
    # создадим функцию для категризации по просрочке
    def delay(row):
        if row['Просрочка, дней']==0:
            return 'Без просрочки'
        elif 1<=row['Просрочка, дней']<=30:
            return 'просрочка 0 - 30 дней'
        elif 31<=row['Просрочка, дней']<=60:
            return 'просрочка 31-60 дней'
        elif 61<=row['Просрочка, дней']<=90:
            return 'просрочка 61-90 дней'
        else:
            return 'просрочка 91+ дней'
        
    # создаем столбец с категориями
    df['Категория просрочки'] = df.apply(delay,axis =1)
    
    # разбиваем датасет на два по годам
    df_2019 = df[df['Дата'].dt.year == 2019] 
    df_2020 = df[df['Дата'].dt.year == 2020]
    
    # так как нам важна динамика изменения просрочек из года в год
    # для анализа возьмем когорту пользователей из 2019 года
    # так как динамика будет видна только для них в 2020 году
    
    # объединяем датасеты по трем столбцам
    df_group = df_2019.merge(
        df_2020,on = [
            'id', 'Тип обязательства','Первоначальный кредитор'
        ]
    )
    # создадим матрицу 
    df_matrix = df_group.pivot_table(
        index='Категория просрочки_y', 
        columns = 'Категория просрочки_x',
        values = 'id',aggfunc = 'count'
    )
    #пропущенные значения заполним нулями
    df_matrix = df_matrix.fillna(0)
    
    # отформатируем нашу таблицу по заданию
    df_matrix = df_matrix.rename_axis('',axis=0)
    df_matrix = df_matrix.rename_axis(bank,axis=1)
    df_matrix = pd.concat({'данные на 01.01.2020': df_matrix})
    df_matrix = pd.concat({'данные на 01.01.2019': df_matrix},axis = 1)
    
    # создадим excel файл и импортируем туда матрицу
    df_matrix.to_excel(
        'Матрица_миграции_просрочек.xlsx',
        sheet_name = 'Матрица'
    )
    
    
    return df_matrix

    
    

In [324]:
# проверим

matrix_func(db_path,'СБЕРБАНК','Ипотека')

данные на 01.01.2019  \
СБЕРБАНК                                          Без просрочки   
                                                                  
данные на 01.01.2020 Без просрочки                        209.0   
                     просрочка 0 - 30 дней                  5.0   
                     просрочка 61-90 дней                   1.0   
                     просрочка 91+ дней                     1.0   

                                                                 \
СБЕРБАНК                                   просрочка 31-60 дней   
                                                                  
данные на 01.01.2020 Без просрочки                          1.0   
                     просрочка 0 - 30 дней                  0.0   
                     просрочка 61-90 дней                   0.0   
                     просрочка 91+ дней                     0.0   

                                                               
СБЕРБАНК                                   просрочка 91+ дней  
                                                               
данные на 01.01.2020 Без просрочки                        2.0  
                     просрочка 0 - 30 дней                0.0  
                     просрочка 61-90 дней                 0.0  
                     просрочка 91+ дней                  23.0